# Try 1

## Data loading

In [27]:
import numpy as np
import tensorflow as tf
from PIL import Image

In [35]:
img_width, img_height = 150, 150
#img_width, img_height = 224, 224


train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

## Small ConvNet

### Model architecture definition

In [3]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    input_layer = tf.reshape(features["x"], [-1, img_width, img_height, 3])
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[3, 3],
        padding='same',
        activation=tf.nn.relu)
    
    # Pooling #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=32,
        kernel_size=[3,3],
        padding='same',
        activation=tf.nn.relu)
    
    # Pooling #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer #3
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=64,
        kernel_size=[3,3],
        padding='same',
        activation=tf.nn.relu)
    
    # Pooling #3
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    
    # Dense Layer
    flatten = tf.contrib.layers.flatten(inputs=pool3)
    dense = tf.layers.dense(inputs=flatten, units=64, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=2)
    
    predicitions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor"),
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predicitions=predicitions)
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32),depth=2)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation metris (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predicitions["classes"])        
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

### Load Training and Test Data

In [29]:
import os
def import_images_from_directory(direcotry, target_size=None, samples=None):
    images = []
    labels = []

    count = 0
    classes = {c: i for i, c in enumerate(os.listdir(direcotry))}
    for c, i in classes.items():
        if samples is not None and count >= samples:
            break
        p = os.path.join(direcotry, c)
        for f in os.listdir(p):
            if samples is not None and count >= samples:
                break
            im = Image.open(os.path.join(p, f))
            if target_size is not None:
                im = resize(im, target_size)
            images.append(np.array(im, dtype="float32")/255)
            im.close()
            labels.append(i)
            count += 1
    return np.array(images), np.array(labels, dtype="float32")

def resize(im, target_size):
    return im.resize(target_size, Image.ANTIALIAS)

In [36]:
train_images, train_labels = import_images_from_directory(train_data_dir, (img_width, img_height))

In [37]:
train_images.shape

(2000, 224, 224, 3)

### Train

In [7]:
# Create the Estimator
classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_mode24")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_mode24', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [8]:
# Set up a Loggin Hook
#tensors_to_log = {"probabilites": "softmax_tensor"}
tensors_to_log = {}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [9]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_images},
    y=train_labels,
    batch_size=16,
    num_epochs=30,
    shuffle=True)

In [10]:
classifier.train(input_fn=train_input_fn, hooks=[logging_hook])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_mode24/model.ckpt.
INFO:tensorflow:
INFO:tensorflow:loss = 0.669343, step = 1
INFO:tensorflow: (0.814 sec)
INFO:tensorflow:global_step/sec: 62.2182
INFO:tensorflow: (0.795 sec)
INFO:tensorflow:loss = 0.659755, step = 101 (1.609 sec)
INFO:tensorflow: (0.801 sec)
INFO:tensorflow:global_step/sec: 62.5511
INFO:tensorflow: (0.798 sec)
INFO:tensorflow:loss = 0.948668, step = 201 (1.599 sec)
INFO:tensorflow: (0.804 sec)
INFO:tensorflow:global_step/sec: 62.9818
INFO:tensorflow: (0.784 sec)
INFO:tensorflow:loss = 0.559279, step = 301 (1.588 sec)
INFO:tensorflow: (0.800 sec)
INFO:tensorflow:global_step/sec: 61.4913
INFO:tensorflow: (0.826 sec)
INFO:tensorflow:loss = 0.568861, step = 401 (1.625 sec)
INFO:tensorflow: (0.798 sec)
INFO:tensorflow:global_step/sec: 62.2512
INFO:tensorflow: (0.809 sec)
INFO:tensorflow:loss = 0.315964, step = 501 (1.606 sec)
INFO:tensorflow: (0.813 sec)
INFO:tens

### Validation

In [13]:
eval_images, eval_labels = import_images_from_directory(validation_data_dir, (150, 150))

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_images},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

In [14]:
eval_results = classifier.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Starting evaluation at 2017-12-21-12:07:23
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_mode24/model.ckpt-3750
INFO:tensorflow:Finished evaluation at 2017-12-21-12:07:23
INFO:tensorflow:Saving dict for global step 3750: accuracy = 0.7225, global_step = 3750, loss = 2.83679


#  Data augmentation for improving the model

In [13]:
train_datagen_augmented = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# same code as before
train_generator_augmented = train_datagen_augmented.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


In [14]:
classifier.train(input_fn=next_wrapper(train_generator_augmented), steps=20000, hooks=[logging_hook])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model2/model.ckpt.
INFO:tensorflow:probabilites = [[ 0.50544655  0.49455351]
 [ 0.50714421  0.4928557 ]
 [ 0.50731665  0.49268329]
 [ 0.50971085  0.49028918]
 [ 0.49844089  0.50155914]
 [ 0.49641463  0.50358534]
 [ 0.51678395  0.48321602]
 [ 0.52028722  0.47971284]
 [ 0.50308001  0.49691993]
 [ 0.49852896  0.50147098]
 [ 0.50439411  0.49560592]
 [ 0.51380295  0.48619711]
 [ 0.50841951  0.49158043]
 [ 0.4911882   0.50881177]
 [ 0.50445992  0.49554014]
 [ 0.50157654  0.49842349]
 [ 0.50585473  0.4941453 ]
 [ 0.50364524  0.49635479]
 [ 0.50474215  0.49525782]
 [ 0.51053494  0.48946506]
 [ 0.49674422  0.50325578]
 [ 0.49961275  0.50038731]
 [ 0.50467247  0.4953275 ]
 [ 0.49249297  0.50750703]
 [ 0.4983871   0.5016129 ]
 [ 0.5062297   0.49377033]
 [ 0.5021131   0.49788687]
 [ 0.51954269  0.48045728]
 [ 0.5194155   0.48058453]
 [ 0.50219321  0.49780682]
 [ 0.51691365  0.48308632]
 [ 0

INFO:tensorflow:loss = 0.6766, step = 401 (3.392 sec)
INFO:tensorflow:probabilites = [[ 0.49350563  0.50649434]
 [ 0.45404929  0.54595077]
 [ 0.46726418  0.53273576]
 [ 0.4885079   0.51149213]
 [ 0.4522886   0.54771137]
 [ 0.46537554  0.53462446]
 [ 0.48973158  0.51026839]
 [ 0.4759658   0.5240342 ]
 [ 0.49759468  0.50240535]
 [ 0.47697017  0.5230298 ]
 [ 0.4695116   0.53048837]
 [ 0.47859672  0.52140325]
 [ 0.49248487  0.50751519]
 [ 0.47969168  0.52030832]
 [ 0.47456285  0.52543712]
 [ 0.491678    0.50832206]
 [ 0.4786256   0.52137434]
 [ 0.46874323  0.53125685]
 [ 0.48895749  0.51104254]
 [ 0.44508541  0.55491459]
 [ 0.45948276  0.54051733]
 [ 0.4783788   0.52162123]
 [ 0.45841071  0.54158932]
 [ 0.47269469  0.52730531]
 [ 0.46754241  0.53245765]
 [ 0.48025149  0.51974851]
 [ 0.49296373  0.50703621]
 [ 0.48965555  0.51034445]
 [ 0.48616806  0.51383191]
 [ 0.50561231  0.49438769]
 [ 0.46324196  0.53675812]
 [ 0.45098144  0.54901862]] (1.696 sec)
INFO:tensorflow:global_step/sec: 29.45

INFO:tensorflow:global_step/sec: 29.4483
INFO:tensorflow:probabilites = [[ 0.50054079  0.49945927]
 [ 0.46812859  0.53187138]
 [ 0.45714286  0.54285711]
 [ 0.47350481  0.52649522]
 [ 0.43163636  0.56836367]
 [ 0.46137056  0.53862941]
 [ 0.48668501  0.51331508]
 [ 0.43539515  0.56460488]
 [ 0.47227189  0.52772814]
 [ 0.50662231  0.49337766]
 [ 0.47174031  0.52825963]
 [ 0.47699454  0.52300537]
 [ 0.46276549  0.53723449]
 [ 0.51088208  0.48911798]
 [ 0.41998732  0.58001268]
 [ 0.48184732  0.51815265]
 [ 0.47212657  0.5278734 ]
 [ 0.4750067   0.5249933 ]
 [ 0.46192339  0.53807664]
 [ 0.43829978  0.56170022]
 [ 0.4639959   0.53600413]
 [ 0.43528864  0.56471133]
 [ 0.47513092  0.52486908]
 [ 0.49365211  0.50634784]
 [ 0.39423928  0.60576069]
 [ 0.45923728  0.54076266]
 [ 0.46748346  0.5325166 ]
 [ 0.45511457  0.5448854 ]
 [ 0.50012845  0.49987161]
 [ 0.49259397  0.50740606]
 [ 0.43676409  0.56323588]
 [ 0.46601549  0.53398448]] (1.698 sec)
INFO:tensorflow:loss = 0.674753, step = 901 (3.396 

INFO:tensorflow:loss = 0.662658, step = 1301 (3.396 sec)
INFO:tensorflow:probabilites = [[ 0.51343477  0.48656517]
 [ 0.4894914   0.5105086 ]
 [ 0.40361145  0.59638852]
 [ 0.53035098  0.46964905]
 [ 0.50618726  0.49381271]
 [ 0.47169271  0.52830732]
 [ 0.48926869  0.51073134]
 [ 0.42425537  0.57574457]
 [ 0.46687877  0.53312123]
 [ 0.51175469  0.48824528]
 [ 0.482797    0.51720297]
 [ 0.47456491  0.52543503]
 [ 0.45288596  0.54711407]
 [ 0.47991794  0.52008212]
 [ 0.42452547  0.5754745 ]
 [ 0.51309514  0.48690489]
 [ 0.49099603  0.50900394]
 [ 0.47074661  0.52925336]
 [ 0.48202038  0.51797968]
 [ 0.39212734  0.60787261]
 [ 0.46967757  0.53032249]
 [ 0.46520284  0.53479719]
 [ 0.47929746  0.5207026 ]
 [ 0.50212991  0.49787015]
 [ 0.4425756   0.55742437]
 [ 0.49928504  0.50071496]
 [ 0.44308847  0.55691153]
 [ 0.48341179  0.51658821]
 [ 0.51372921  0.48627082]
 [ 0.49094048  0.50905955]
 [ 0.43895435  0.56104565]
 [ 0.4415946   0.5584054 ]] (1.697 sec)
INFO:tensorflow:global_step/sec: 29

INFO:tensorflow:global_step/sec: 29.4383
INFO:tensorflow:probabilites = [[ 0.5703426   0.4296574 ]
 [ 0.47448319  0.52551681]
 [ 0.44011787  0.55988216]
 [ 0.54308599  0.45691404]
 [ 0.38799572  0.61200428]
 [ 0.44489607  0.55510396]
 [ 0.47570872  0.52429134]
 [ 0.52992225  0.47007778]
 [ 0.40470716  0.59529281]
 [ 0.560287    0.43971297]
 [ 0.5089972   0.4910028 ]
 [ 0.48099163  0.51900834]
 [ 0.4215292   0.57847077]
 [ 0.5587483   0.44125164]
 [ 0.49485391  0.50514609]
 [ 0.5448463   0.45515367]
 [ 0.50403082  0.49596915]
 [ 0.47331578  0.52668422]
 [ 0.44263545  0.55736458]
 [ 0.40889475  0.59110522]
 [ 0.42346215  0.57653791]
 [ 0.44981843  0.55018163]
 [ 0.47760451  0.52239543]
 [ 0.48830068  0.51169932]
 [ 0.38533932  0.61466074]
 [ 0.4258599   0.57414007]
 [ 0.49396592  0.50603402]
 [ 0.44546247  0.55453753]
 [ 0.53664815  0.46335191]
 [ 0.50978786  0.49021217]
 [ 0.46757919  0.53242087]
 [ 0.47440249  0.52559745]] (1.696 sec)
INFO:tensorflow:loss = 0.625642, step = 1801 (3.397

...

INFO:tensorflow:global_step/sec: 29.4968
INFO:tensorflow:probabilites = [[ 0.73077583  0.26922414]
 [ 0.26901451  0.73098546]
 [ 0.26901296  0.73098701]
 [ 0.7310586   0.26894143]
 [ 0.28227097  0.71772909]
 [ 0.26953503  0.73046494]
 [ 0.73005533  0.26994467]
 [ 0.72343057  0.2765694 ]
 [ 0.28755641  0.71244359]
 [ 0.73105842  0.26894158]
 [ 0.73105854  0.26894143]
 [ 0.73101258  0.26898742]
 [ 0.26894155  0.73105842]
 [ 0.7310586   0.26894143]
 [ 0.73100591  0.26899409]
 [ 0.7310586   0.26894143]
 [ 0.73105669  0.26894334]
 [ 0.73105472  0.26894528]
 [ 0.27175438  0.72824562]
 [ 0.26896363  0.73103637]
 [ 0.26894423  0.7310558 ]
 [ 0.268942    0.73105794]
 [ 0.26895812  0.73104191]
 [ 0.7310586   0.26894143]
 [ 0.2691173   0.73088276]
 [ 0.26894224  0.7310577 ]
 [ 0.73104304  0.26895699]
 [ 0.7306937   0.2693063 ]
 [ 0.73104966  0.26895028]
 [ 0.26961756  0.7303825 ]
 [ 0.26894429  0.73105574]
 [ 0.2689417   0.73105836]] (1.697 sec)
INFO:tensorflow:loss = 0.315237, step = 18001 (3.39

INFO:tensorflow:loss = 0.315014, step = 18401 (3.390 sec)
INFO:tensorflow:probabilites = [[ 0.7310586   0.26894143]
 [ 0.26894143  0.7310586 ]
 [ 0.26894152  0.73105848]
 [ 0.73105794  0.26894197]
 [ 0.27017212  0.72982788]
 [ 0.26925772  0.73074228]
 [ 0.73058963  0.26941037]
 [ 0.7304672   0.26953277]
 [ 0.27214599  0.72785401]
 [ 0.73105764  0.26894236]
 [ 0.73105735  0.26894271]
 [ 0.73005986  0.26994014]
 [ 0.2689417   0.7310583 ]
 [ 0.73024476  0.26975527]
 [ 0.72861576  0.27138424]
 [ 0.73060769  0.26939231]
 [ 0.73101056  0.26898944]
 [ 0.73105854  0.26894143]
 [ 0.26894155  0.73105848]
 [ 0.26895177  0.73104823]
 [ 0.26894143  0.7310586 ]
 [ 0.26894143  0.7310586 ]
 [ 0.26895764  0.73104239]
 [ 0.73105782  0.26894221]
 [ 0.26894146  0.73105854]
 [ 0.2689417   0.73105836]
 [ 0.73080832  0.26919174]
 [ 0.73100638  0.26899368]
 [ 0.72965246  0.27034754]
 [ 0.26894999  0.73105007]
 [ 0.26894149  0.73105848]
 [ 0.26895079  0.73104918]] (1.696 sec)
INFO:tensorflow:global_step/sec: 2

INFO:tensorflow:global_step/sec: 29.4626
INFO:tensorflow:probabilites = [[ 0.73105836  0.26894167]
 [ 0.26894143  0.73105854]
 [ 0.26911578  0.73088419]
 [ 0.7310586   0.26894143]
 [ 0.26945242  0.73054755]
 [ 0.26894143  0.73105854]
 [ 0.73027599  0.26972404]
 [ 0.68637216  0.31362781]
 [ 0.26894405  0.73105592]
 [ 0.72875988  0.27124009]
 [ 0.73105854  0.26894143]
 [ 0.73102796  0.26897201]
 [ 0.26917043  0.7308296 ]
 [ 0.73075163  0.26924837]
 [ 0.71292758  0.28707239]
 [ 0.73105234  0.2689476 ]
 [ 0.73102093  0.26897904]
 [ 0.73050851  0.26949149]
 [ 0.26894283  0.73105723]
 [ 0.26902467  0.73097527]
 [ 0.26894155  0.73105842]
 [ 0.26894167  0.73105836]
 [ 0.26896656  0.73103344]
 [ 0.73105842  0.26894158]
 [ 0.26894164  0.73105836]
 [ 0.26931962  0.73068035]
 [ 0.73073369  0.26926628]
 [ 0.73104912  0.26895091]
 [ 0.73103863  0.2689614 ]
 [ 0.2689679   0.73103207]
 [ 0.27027836  0.72972167]
 [ 0.26894146  0.73105854]] (1.698 sec)
INFO:tensorflow:loss = 0.316323, step = 18901 (3.39

INFO:tensorflow:loss = 0.315721, step = 19301 (3.391 sec)
INFO:tensorflow:probabilites = [[ 0.7310583   0.26894173]
 [ 0.26894143  0.7310586 ]
 [ 0.26894203  0.73105794]
 [ 0.7310586   0.26894143]
 [ 0.26895562  0.73104441]
 [ 0.26894206  0.73105788]
 [ 0.7310586   0.26894143]
 [ 0.73102874  0.26897126]
 [ 0.2689473   0.7310527 ]
 [ 0.73105544  0.26894462]
 [ 0.73105079  0.26894927]
 [ 0.73105854  0.26894143]
 [ 0.269043    0.73095691]
 [ 0.73105723  0.26894277]
 [ 0.73105115  0.26894882]
 [ 0.70937347  0.29062656]
 [ 0.73105818  0.26894188]
 [ 0.73064256  0.26935744]
 [ 0.26894149  0.73105854]
 [ 0.26894245  0.73105758]
 [ 0.26895496  0.73104513]
 [ 0.26894206  0.73105794]
 [ 0.32626736  0.6737327 ]
 [ 0.73104239  0.26895767]
 [ 0.26895392  0.73104608]
 [ 0.26898077  0.73101926]
 [ 0.73080635  0.26919368]
 [ 0.73105848  0.26894155]
 [ 0.72996646  0.27003354]
 [ 0.26914391  0.73085612]
 [ 0.27442932  0.72557062]
 [ 0.26894143  0.7310586 ]] (1.697 sec)
INFO:tensorflow:global_step/sec: 2

INFO:tensorflow:global_step/sec: 29.455
INFO:tensorflow:probabilites = [[ 0.7310586   0.26894143]
 [ 0.26894143  0.7310586 ]
 [ 0.26894164  0.73105836]
 [ 0.7310586   0.26894143]
 [ 0.26981464  0.73018539]
 [ 0.26894143  0.7310586 ]
 [ 0.7310586   0.26894143]
 [ 0.66322517  0.33677483]
 [ 0.26894557  0.73105448]
 [ 0.7310586   0.26894143]
 [ 0.73100454  0.26899552]
 [ 0.73008883  0.26991123]
 [ 0.26894149  0.73105848]
 [ 0.73105723  0.26894277]
 [ 0.7310558   0.26894417]
 [ 0.73105776  0.26894221]
 [ 0.7310583   0.26894173]
 [ 0.73105842  0.26894158]
 [ 0.26894158  0.73105842]
 [ 0.27007738  0.72992259]
 [ 0.26894221  0.73105776]
 [ 0.26894143  0.7310586 ]
 [ 0.26894143  0.7310586 ]
 [ 0.73081863  0.26918134]
 [ 0.26894757  0.73105252]
 [ 0.26894152  0.73105848]
 [ 0.69845039  0.30154955]
 [ 0.73105597  0.26894403]
 [ 0.7310586   0.26894143]
 [ 0.26894277  0.73105723]
 [ 0.26894361  0.73105645]
 [ 0.26894143  0.7310586 ]] (1.699 sec)
INFO:tensorflow:loss = 0.317872, step = 19801 (3.395

In [15]:
datagen = ImageDataGenerator(rescale=1./255)

validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

Found 800 images belonging to 2 classes.


In [18]:
eval_results = classifier.evaluate(input_fn=next_wrapper(validation_generator), steps=validation_generator.n / 32)

INFO:tensorflow:Starting evaluation at 2017-12-16-06:40:12
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model2/model.ckpt-20000
INFO:tensorflow:Evaluation [1/25]
INFO:tensorflow:Evaluation [2/25]
INFO:tensorflow:Evaluation [3/25]
INFO:tensorflow:Evaluation [4/25]
INFO:tensorflow:Evaluation [5/25]
INFO:tensorflow:Evaluation [6/25]
INFO:tensorflow:Evaluation [7/25]
INFO:tensorflow:Evaluation [8/25]
INFO:tensorflow:Evaluation [9/25]
INFO:tensorflow:Evaluation [10/25]
INFO:tensorflow:Evaluation [11/25]
INFO:tensorflow:Evaluation [12/25]
INFO:tensorflow:Evaluation [13/25]
INFO:tensorflow:Evaluation [14/25]
INFO:tensorflow:Evaluation [15/25]
INFO:tensorflow:Evaluation [16/25]
INFO:tensorflow:Evaluation [17/25]
INFO:tensorflow:Evaluation [18/25]
INFO:tensorflow:Evaluation [19/25]
INFO:tensorflow:Evaluation [20/25]
INFO:tensorflow:Evaluation [21/25]
INFO:tensorflow:Evaluation [22/25]
INFO:tensorflow:Evaluation [23/25]
INFO:tensorflow:Evaluation [24/25]
INFO:tensorflow:Evaluatio

# Using a pre-trained model

In [46]:
import os, time

import numpy as np
import tensorflow as tf

VGG_MEAN = [103.939, 116.779, 123.68]

class Vgg16:
    def __init__(self, vgg16_npy_path=None):
        if vgg16_npy_path is None:
            path = os.path.join('.', "vgg16.npy")
            vgg16_npy_path = path
            print(path)
        
        self.data_dict = np.load(vgg16_npy_path, encoding='latin1').item()
        print("npy file loaded")
    
    def build(self, rgb):
        """
        load variable from npy to build the VGG
        :param rgb: rgb image [batch, height, width, 3] values scaled [0, 1]
        """
        start_time = time.time()
        print("build model started")
        rgb_scaled = rgb * 255.0
        
        # Convert RGB to BGR
        red, green, blue = tf.split(axis=3, num_or_size_splits=3, value=rgb_scaled)
        
        bgr = tf.concat(axis=3, values=[
            blue - VGG_MEAN[0],
            green - VGG_MEAN[1],
            red - VGG_MEAN[2],
        ])
        
        self.conv1_1 = self.conv_layer(bgr, "conv1_1")
        self.conv1_2 = self.conv_layer(self.conv1_1, "conv1_2")
        self.pool1 = self.max_pool(self.conv1_2, "pool1")
        
        self.conv2_1 = self.conv_layer(self.pool1, "conv2_1")
        self.conv2_2 = self.conv_layer(self.conv2_1, "conv2_2")
        self.pool2 = self.max_pool(self.conv2_2, "pool2")
        
        self.conv3_1 = self.conv_layer(self.pool2, "conv3_1")
        self.conv3_2 = self.conv_layer(self.conv3_1, "conv3_2")
        self.conv3_3 = self.conv_layer(self.conv3_2, "conv3_3")
        self.pool3 = self.max_pool(self.conv3_3, "pool3")
        
        self.conv4_1 = self.conv_layer(self.pool3, "conv4_1")
        self.conv4_2 = self.conv_layer(self.conv4_1, "conv4_2")
        self.conv4_3 = self.conv_layer(self.conv4_2, "conv4_3")
        self.pool4 = self.max_pool(self.conv4_3, "pool4")
        
        self.conv5_1 = self.conv_layer(self.pool4, "conv5_1")
        self.conv5_2 = self.conv_layer(self.conv5_1, "conv5_2")
        self.conv5_3 = self.conv_layer(self.conv5_2, "conv5_3")
        self.pool5 = self.max_pool(self.conv5_3, "pool5")
        '''
        self.fc6 = self.fc_layer(self.pool5, "fc6")
        self.relu6 = tf.nn.relu(self.fc6)
        
        self.fc7 = self.fc_layer(self.relu6, "fc7")
        self.relu7 = tf.nn.relu(self.fc7)
        
        self.fc8 = self.fc_layer(self.relu7, "fc8")
        '''
        self.flatten = tf.contrib.layers.flatten(inputs=self.pool5)
        self.dense6 = tf.layers.dense(self.flatten, 256, activation=tf.nn.relu)
        self.keep_prob = tf.placeholder(tf.float32)
        self.dropout7 = tf.nn.dropout(self.dense6, self.keep_prob)
        
        self.y_conv = tf.layers.dense(self.dropout7, 2)        
        #self.prob = tf.nn.softmax(self.dense8, name="prob")
        
        self.data_dict = None
        print(("build model finished: %ds" % (time.time() - start_time)))
        
    def conv_layer(self, bottom, name):
        with tf.variable_scope(name):
            filt = self.get_conv_filter(name)
            conv = tf.nn.conv2d(bottom, filt, [1,1,1,1], padding='SAME')
            
            conv_biases = self.get_bias(name)
            bias = tf.nn.bias_add(conv, conv_biases)
            
            return tf.nn.relu(bias)
    
    def max_pool(slef, bottom, name):
        return tf.nn.max_pool(bottom, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name=name)
    
    def fc_layer(self, bottom, name):
        with tf.variable_scope(name):
            shape = bottom.get_shape().as_list()
            dim = 1
            for d in shape[1:]:
                dim *= d
            x = tf.reshape(bottom, [-1, dim])
            
            weights = self.get_fc_weight(name)
            biases = self.get_bias(name)
            
            # Fully connected layer. Note that the '+' operation automatically
            # broadcasts the biases.
            return  tf.nn.bias_add(tf.matmul(x, weights), biases)
        
    def avg_pool(self, bottom, name):
        return tf.nn.avg_pool(bottom, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name=name)
    
    def get_conv_filter(self, name):
        return tf.constant(self.data_dict[name][0], name="filter")
    
    def get_bias(self, name):
        return tf.constant(self.data_dict[name][1], name='biases')
    
    def get_fc_weight(self, name):
        return tf.constant(self.data_dict[name][0], name='weights')
        

In [45]:
images = tf.placeholder(tf.float32, [None, img_height,img_width,3])
labels = tf.placeholder(tf.float32, [None, 2])
#feed_dict = {images: train_images, labels: train_labels}
vgg = Vgg16()
with tf.name_scope("content_vgg"):
    vgg.build(images)
    
with tf.name_scope("loss"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
        labels=labels,
        logits=vgg.y_conv)
cross_entropy = tf.reduce_mean(cross_entropy)
    
with tf.name_scope('rmspo_optimizer'):
    train_step = tf.train.RMSPropOptimizer(learning_rate=0.001)
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(vgg.y_conv, 1), tf.argmax(labels, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)

batch_size = 50
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        
    
        


./vgg16.npy
npy file loaded
build model started
build model finished: 0s


ValueError: GraphDef cannot be larger than 2GB.

In [12]:
print(vgg16.data_dict.keys())

dict_keys(['conv5_1', 'fc6', 'conv5_3', 'conv5_2', 'fc8', 'fc7', 'conv4_1', 'conv4_2', 'conv4_3', 'conv3_3', 'conv3_2', 'conv3_1', 'conv1_1', 'conv1_2', 'conv2_2', 'conv2_1'])


In [14]:
print('conv1_1[0]:', np.shape(vgg16.data_dict['conv1_1'][0]))
print('conv1_1[1]:', np.shape(vgg16.data_dict['conv1_1'][1]))
print('conv1_2[0]:', np.shape(vgg16.data_dict['conv1_2'][0]))
print('conv1_2[1]:', np.shape(vgg16.data_dict['conv1_2'][1]))

conv1_1[0]: (3, 3, 3, 64)
conv1_1[1]: (64,)
conv1_2[0]: (3, 3, 64, 64)
conv1_2[1]: (64,)


In [15]:
print('conv2_1[0]:', np.shape(vgg16.data_dict['conv2_1'][0]))
print('conv2_1[1]:', np.shape(vgg16.data_dict['conv2_1'][1]))
print('conv2_2[0]:', np.shape(vgg16.data_dict['conv2_2'][0]))
print('conv2_2[1]:', np.shape(vgg16.data_dict['conv2_2'][1]))

conv2_1[0]: (3, 3, 64, 128)
conv2_1[1]: (128,)
conv2_2[0]: (3, 3, 128, 128)
conv2_2[1]: (128,)


In [17]:
print('conv3_1[0]:', np.shape(vgg16.data_dict['conv3_1'][0]))
print('conv3_1[1]:', np.shape(vgg16.data_dict['conv3_1'][1]))
print('conv3_2[0]:', np.shape(vgg16.data_dict['conv3_2'][0]))
print('conv3_2[1]:', np.shape(vgg16.data_dict['conv3_2'][1]))
print('conv3_3[0]:', np.shape(vgg16.data_dict['conv3_3'][0]))
print('conv3_3[1]:', np.shape(vgg16.data_dict['conv3_3'][1]))

conv3_1[0]: (3, 3, 128, 256)
conv3_1[1]: (256,)
conv3_2[0]: (3, 3, 256, 256)
conv3_2[1]: (256,)
conv3_3[0]: (3, 3, 256, 256)
conv3_3[1]: (256,)


In [18]:
print('conv4_1[0]:', np.shape(vgg16.data_dict['conv4_1'][0]))
print('conv4_1[1]:', np.shape(vgg16.data_dict['conv4_1'][1]))
print('conv4_2[0]:', np.shape(vgg16.data_dict['conv4_2'][0]))
print('conv4_2[1]:', np.shape(vgg16.data_dict['conv4_2'][1]))
print('conv4_3[0]:', np.shape(vgg16.data_dict['conv4_3'][0]))
print('conv4_3[1]:', np.shape(vgg16.data_dict['conv4_3'][1]))

conv4_1[0]: (3, 3, 256, 512)
conv4_1[1]: (512,)
conv4_2[0]: (3, 3, 512, 512)
conv4_2[1]: (512,)
conv4_3[0]: (3, 3, 512, 512)
conv4_3[1]: (512,)


In [22]:
print('conv5_1[0]:', np.shape(vgg16.data_dict['conv5_1'][0]))
print('conv5_1[1]:', np.shape(vgg16.data_dict['conv5_1'][1]))
print('conv5_2[0]:', np.shape(vgg16.data_dict['conv5_2'][0]))
print('conv5_2[1]:', np.shape(vgg16.data_dict['conv5_2'][1]))
print('conv5_3[0]:', np.shape(vgg16.data_dict['conv5_3'][0]))
print('conv5_3[1]:', np.shape(vgg16.data_dict['conv5_3'][1]))

conv5_1[0]: (3, 3, 512, 512)
conv5_1[1]: (512,)
conv5_2[0]: (3, 3, 512, 512)
conv5_2[1]: (512,)
conv5_3[0]: (3, 3, 512, 512)
conv5_3[1]: (512,)


In [24]:
print('fc6[0]:', np.shape(vgg16.data_dict['fc6'][0]))
print('fc6[1]:', np.shape(vgg16.data_dict['fc6'][1]))
print('fc7[0]:', np.shape(vgg16.data_dict['fc7'][0]))
print('fc7[1]:', np.shape(vgg16.data_dict['fc7'][1]))
print('fc8[0]:', np.shape(vgg16.data_dict['fc8'][0]))
print('fc8[1]:', np.shape(vgg16.data_dict['fc8'][1]))

fc6[0]: (25088, 4096)
fc6[1]: (4096,)
fc7[0]: (4096, 4096)
fc7[1]: (4096,)
fc8[0]: (4096, 1000)
fc8[1]: (1000,)
